# Identity Mappings in Deep Residual Networks

Identity Mappings in Deep Residual Networks 논문에서는 새로운 Residual block을 제안하였습니다. <br>
이 논문에서는 기존의 Residual block에서의 처리 순서를 바꾸었습니다. 상세 내용은 아래와 같습니다. <br>

![1](https://i.imgur.com/lgzVx9f.png)


## Abstract ##
skip connection과 after-addition activation을 통하여 identity mapping을 사용하면, 하나의 block에서 다른 block들로  signal들이 forward/backward로 전파됩니다.


## 1. Introduction ##

ResNet은 "Residual Units"들의 누적(stack)들로 구성이 되어 있고 식으로 표현하면 다음과 같습니다.
$$ y_{l} = h(x_{l}) + F(x_{l}, W_{l}) $$

$$ x_{l+1} = f(y_{l}) $$

여기서 $x_{l}$ 과 $x_{l+1} $은 l 번째 layer의 input과 output에 해당합니다. 식의 F는 residual 함수에 해당합니다. $h(x_{l}) = x_{l}$ 식은 identity mapping이고 f은 ReLU 함수 입니다. <br>
100 layer가 넘는 ResNet은 높은 정확도로 몇몇 대회에서 좋은 정확도를 얻었습니다.
ResNet의 핵심 idea는 각각의 $h(x_{l})$에 대하여 residual function 을 더하여 학습하는 것입니다. 여기서 identity mapping $h(x_{l}) = x_{l}$ 을 사용하는 것이 핵심입니다. identity mapping을 추가하는 것을 통하여 identity skip connection("shortcut")이 만들어 집니다.

이 논문을 통하여 볼 점은, 정보의 전파를 위한 "direct path"를 만드는 것에 초점을 두어 Deep Residual Network를 분석하는 것입니다. 여기서 분석할 대상은 residual unit과 더불어 전체 network 입니다.
만약 $h(x_{l})$과 $f(y_{l})$ 둘 다 identity mapping 이라면, signal들은 하나의 unit에서 다른 unit들로 forward/backward 둘 다 직접적으로 전파됩니다.

skip connection의 역할을 이해하기 위해서는, 다양한 타입의 $h(x_{l})$을 비교하고 분석해야 합니다. identity mapping $h(x_{l}) = x_{l}$은 가장 빠른 error reduction과 가장 낮은 training loss를 얻습니다. 반면에 skip connections of scaling, gating 그리고 1×1 convolutions 등은 identity mapping에 비하여 높은 training loss와 error loss를 가집니다.
즉, information path에 어떤 작업도 취하지 않고 깨끗하게 남겨두는 것이 학습하는 데 좋습니다.

identity mapping $f(y_{l}) = y_{l}$을 만들기 위해서는 ReLU나 BN과 같은 activation을 weight layer 이전에 둡니다(pre-activation). weight 연산 이후에 activation을 적용한 것 (post-activation)과는 대조적입니다.
![1](https://i.imgur.com/ffUp8zJ.png)

identity mapping을 이용하면 위와 같이 CIFAR-10/100으로 1001-layer를 쌓았을 때 위와 같이 좋은 성능을 보이며 이러한 성능은 최근 Deep learning의 핵심인 network depth의 dimension을 활용할 여지가 있음을 보여줍니다.

## 2. Analysis of Deep Residual Networks

ResNet은 동일한 연결 형태의 빌딩 block을 stack하는 모듈화 된 아키텍처입니다. 이 block을 "Residual Unit"라고 합니다. <span class="mark">original Residual Unit</span>은 다음과 같이 동작합니다.

$$y_{l} = h(x_{l}) + F(x_{l}, W_{l})$$

$$x_{l+1} = f(y_{l})$$

여기서 $x_{l}$은 l번째 Residual Unit의 input에 해당합니다. $W_{l} = $ { $W_{l,k}$ | $1\leq k \leq K$} 는 weight들(+bias)의 집합입니다. $W_{l,k}$ 에서 l은 l번째 layer를 뜻하고 K는 Residual Unit 내에서의 layer의 숫자를 뜻합니다. 위의 예제에서의 Residual Unit은 K = 2 또는 3이 됩니다.
F는 residual funtion을 뜻합니다. 예를 들어서 residual function은 2개의 3 x 3 convolution layer 결합이 될 수 있습니다. (아래는 Residual Net 초기 논문)

![2](https://i.imgur.com/ZsWyJiI.png)

![3](https://i.imgur.com/HuDEPQL.png)

[초기 논문의 Reisudal Unit 내용 참조]

함수 f는 element-wise addition 연산 이후 적용하는 ReLU 함수 입니다. 함수 h는 identity mapping으로 $h(x_{l}) = x_{l}$에 해당합니다.

<span class="mark">만약에 f가 ReLU가 아닌 identity mapping 이라고 가정</span>해 봅시다. 그러면 $ x_{l+1} \equiv y_{l} $식이 성립합니다. 그러면 다음과 같은 식이 성립합니다.

$$ x_{l+1} = x_{l} + F(x_{l}, W_{l}) $$

Recursive 하게 접근해 보면, $x_{l+2} = x_{l+1} + F(x_{l+1}, W_{l+1}) = x_{l} + F(x_{l}, W_{l}) + F(x_{l+1}, W_{l+1})$ 가 됩니다. 따라서,

$$ x_{L} = x_{l} + \Sigma_{i = l}^{L-1} F(x_{i}, W_{i}) $$
    
여기서 l은 얕은 layer 단계이고 L은 깊은 layer 단계입니다. 따라서 L > l 이 됩니다.

f가 ReLU가 아닌 identity mapping 이라고 가정하였을 때, 위와 같은 식으로 정리할 수 있고 아래와 같은 2가지 특성이 생깁니다.

- L 단계의 깊은 layer 단계를 l 단계의 앝은 layer로 표현할 수 있습니다. 이 때 $\Sigma_{i = l}^{L-1} F$ 식을 이용합니다. 
- $ 깊은 layer L에 대하여 x_{L} = x_{0} + \Sigma^{L-1}_{i=0}F(x_{i}, W_{i})$ 형태의 summation으로 L 단계 이전을 residual function으로 표현할 수 있습니다.